In [10]:
import boto3
import pandas as pd
import numpy as np
from copy import deepcopy as cp
import json
from pprint import pprint
import time

In [11]:
aws_profile = 'peter-personal'
session = boto3.Session(profile_name='peter-personal')

In [12]:
# Read in the configuration Files
with open('FlowerTypes.json') as json_data:
    flower_types_dict = json.load(json_data)

with open('OrderFlowers.json') as json_data:
    order_flowers_dict = json.load(json_data)
    
with open('OrderFlowersBot.json') as json_data:
    order_flowers_bot_dict = json.load(json_data)

In [44]:
order_flowers_bot_dict['name']

'OrderFlowersBot'

In [45]:
# Make Bot
model_builder = session.client('lex-models')
put_slot_out = model_builder.put_slot_type(**flower_types_dict)
put_intent_out = model_builder.put_intent(**order_flowers_dict)
put_bot_out = model_builder.put_bot(**order_flowers_bot_dict)

In [46]:
# Wait Untill Bot is Built
def check_bot_status(model_builder):
    response_dict = model_builder.get_bot(name='OrderFlowersBot',
                                          versionOrAlias='$LATEST')
    status = response_dict['status']
    print("Bot status {}".format(status))
    return status
    
status = check_bot_status(model_builder)

while(status!='READY'):
    time.sleep(5)
    status = check_bot_status(model_builder)

Bot status BUILDING
Bot status BUILDING
Bot status BUILDING
Bot status BUILDING
Bot status BUILDING
Bot status BUILDING
Bot status READY


In [ ]:
# I've manually added the dev alias for now
# I'd have to delete the channel, then delete the alias, then delete the bot

In [42]:
model_builder.create_bot_version?

In [57]:
put_bot_alias_out = model_builder.put_bot_alias(botName=order_flowers_bot_dict['name'],
                                                     botVersion='$LATEST',
                                                     name='dev')

PreconditionFailedException: An error occurred (PreconditionFailedException) when calling the PutBotAlias operation: Alias 'dev' for Bot 'OrderFlowersBot' already exists. If you are trying to update 'dev' you must specify checksum

In [56]:
create_bot_version_out

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '200',
   'content-type': 'application/json',
   'date': 'Wed, 21 Mar 2018 14:24:44 GMT',
   'x-amzn-requestid': '9a269be4-2d13-11e8-8f29-9915df4142a2'},
  'HTTPStatusCode': 200,
  'RequestId': '9a269be4-2d13-11e8-8f29-9915df4142a2',
  'RetryAttempts': 0},
 'botName': 'OrderFlowersBot',
 'botVersion': '$LATEST',
 'checksum': 'a6a95322-4424-41ac-8833-e8c69932f1ff',
 'createdDate': datetime.datetime(2018, 3, 21, 10, 24, 44, 871000, tzinfo=tzlocal()),
 'lastUpdatedDate': datetime.datetime(2018, 3, 21, 10, 24, 44, 871000, tzinfo=tzlocal()),
 'name': 'dev'}

In [51]:
lex_runtime = session.client('lex-runtime')

In [6]:
# Interact with Bot
lex_runtime = session.client('lex-runtime')
out = lex_runtime.post_text(botName = 'OrderFlowersBot',
                      botAlias = bot_alias,
                      userId = "UserOne",
                      inputText = "i would like to order flowers" )
pprint(out)

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '279',
                                      'content-type': 'application/json',
                                      'date': 'Tue, 20 Mar 2018 19:03:23 GMT',
                                      'x-amzn-requestid': '5d0d9de4-2c71-11e8-b202-c1f3d18f6e51'},
                      'HTTPStatusCode': 200,
                      'RequestId': '5d0d9de4-2c71-11e8-b202-c1f3d18f6e51',
                      'RetryAttempts': 0},
 'dialogState': 'ElicitSlot',
 'intentName': 'OrderFlowers',
 'message': 'What type of flowers would you like to order?',
 'messageFormat': 'PlainText',
 'slotToElicit': 'FlowerType',
 'slots': {'FlowerType': None, 'PickupDate': None, 'PickupTime': None}}


In [22]:
# Delete the Bot

# TODO make this more robust to error handling
model_builder.delete_bot(name='OrderFlowersBot')
model_builder.delete_intent(name='OrderFlowers')
model_builder.delete_slot_type(name='FlowerTypes')

ConflictException: An error occurred (ConflictException) when calling the DeleteIntent operation: There is a conflicting operation in progress for the resource named 'OrderFlowers'.

In [ ]:
# Build a bot cleaner https://docs.aws.amazon.com/lex/latest/dg/gs-cli-clean-up.html
# Build Testing service for bot
# Build some simple gui for it / Interact with someone else
# 